In [1]:
import json
from tqdm import tqdm
from pathlib import Path
from timeit import default_timer as timer

import pandas as pd
import numpy as np
from io import BytesIO
from PIL import Image
import h5py

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from timm import create_model

from accelerate import Accelerator

from isic_helper import DotDict
from isic_helper import get_folds
from isic_helper import time_to_str

In [4]:
INPUT_PATH = Path("../input/isic-2024-challenge/")
ARTIFACTS_INPUT_PATH = Path(f"../input/isic-scd-resnet18-train/")

with open(ARTIFACTS_INPUT_PATH / "run_metadata.json", "r") as f:
    run_metadata = json.load(f)

cfg = DotDict()
for k, v in run_metadata["params"].items():
    setattr(cfg, k, v)
setattr(cfg, "infer", True)
print(cfg)

MODELS_INPUT_PATH = ARTIFACTS_INPUT_PATH / cfg.models_output_dir

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False)

test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv", low_memory=False)
test_images = h5py.File(INPUT_PATH / "test-image.hdf5", mode="r")

folds_df = get_folds()
train_metadata = train_metadata.merge(folds_df, on=["isic_id", "patient_id"], how="inner")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

DotDict(_temp=False, _key=None, _parent=None, infer=True, cpu=False, mixed_precision='fp16', tta=True, pos_ratio=0.1, image_size=64, lr=0.0005, num_epochs=1, seed=2022, train_batch_size=256, train_num_worker=2, val_batch_size=256, val_num_worker=2, log_every=10, models_output_dir='models', model_name='resnet18_v1')
Train data size: (401059, 57)
Test data size: (3, 44)


In [5]:
id_column = "isic_id"
target_column = "target"
folds = train_metadata["fold"]

In [6]:
accelerator = Accelerator(cpu=cfg.cpu, mixed_precision=cfg.mixed_precision)

In [7]:
accelerator.device

device(type='cuda')

In [8]:
def test_augment(image_size):
    transform = A.Compose([
        A.Resize(image_size, image_size),
#         A.Normalize(
#             mean=[0., 0., 0.],
#             std=[1, 1, 1],
#             max_pixel_value=255.0,
#             p=1.0
#         ),
        ToTensorV2()
    ], p=1.)
    return transform

class ISICDataset(Dataset):
    def __init__(self, metadata, images, augment, infer=False):
        self.metadata = metadata
        self.images = images
        self.augment = augment
        self.length = len(self.metadata)
        self.infer = infer
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        data = self.metadata.iloc[index]
        
        image = np.array(Image.open(BytesIO(self.images[data[id_column]][()])))
        image = self.augment(image=image)["image"]
        
        record = {
            "image": image
        }
        
        if not self.infer:
            target = data[target_column]
            record["target"] = torch.tensor(target).float()
        
        return record

class ISICNet(nn.Module):
    def __init__(self, arch="resnet18", pretrained=False, infer=False):
        super(ISICNet, self).__init__()
        self.infer = infer
        self.model = create_model(model_name=arch, pretrained=pretrained, in_chans=3,  num_classes=0, global_pool='')
        self.classifier = nn.Linear(self.model.num_features, 1)
        
        self.dropouts = nn.ModuleList([nn.Dropout(0.5) for i in range(5)])
        
    def forward(self, batch):
        image = batch["image"]
        image = image.float() / 255
        
        x = self.model(image)
        bs = len(image)
        pool = F.adaptive_avg_pool2d(x, 1).reshape(bs,-1)
        
        if self.training:
            logit = 0
            for i in range(len(self.dropouts)):
                logit += self.classifier(self.dropouts[i](pool))
            logit = logit/len(self.dropouts)
        else:
            logit = self.classifier(pool)
        return logit

In [12]:
all_folds = np.sort(folds.unique())
test_predictions_df = pd.DataFrame({id_column: test_metadata[id_column]})
test_dataset = ISICDataset(test_metadata, test_images, augment=test_augment(image_size=cfg.image_size), infer=True)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=cfg.val_batch_size, num_workers=cfg.val_num_worker, drop_last=False, pin_memory=True)
for fold in all_folds:
    net = ISICNet(pretrained=False, infer=True)
    net = net.to(accelerator.device)
    
    net, test_dataloader = accelerator.prepare(net, test_dataloader)
    
    accelerator.load_state(MODELS_INPUT_PATH / 
                           f"fold_{fold}/model_{cfg.model_name}_epoch_{run_metadata['best_num_epochs'][f'fold_{fold}']}")
    
    net.eval()
    test_preds = []
    for step, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        # We could avoid this line since we set the accelerator with `device_placement=True`.
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}
        
        image0 = batch['image'].clone().detach()
        test_preds_batch = 0
        counter = 0
        with torch.no_grad():
            outputs = net(batch)
        preds = torch.sigmoid(outputs)
        preds = accelerator.gather_for_metrics((preds))
        test_preds_batch += preds.data.cpu().numpy().reshape(-1)
        counter += 1
        if cfg.tta:
            batch["image"] = torch.flip(image0,dims=[2])
            with torch.no_grad():
                outputs = net(batch)
            preds = torch.sigmoid(outputs)
            preds = accelerator.gather_for_metrics((preds))
            test_preds_batch += preds.data.cpu().numpy().reshape(-1)
            counter += 1

            batch["image"] = torch.flip(image0,dims=[3])
            with torch.no_grad():
                outputs = net(batch)
            preds = torch.sigmoid(outputs)
            preds = accelerator.gather_for_metrics((preds))
            test_preds_batch += preds.data.cpu().numpy().reshape(-1)
            counter += 1

            for k in [1, 2, 3]:
                batch["image"] = torch.rot90(image0,k, dims=[2, 3])
                with torch.no_grad():
                    outputs = net(batch)
                preds = torch.sigmoid(outputs)
                preds = accelerator.gather_for_metrics((preds))
                test_preds_batch += preds.data.cpu().numpy().reshape(-1)
                counter += 1
                
        test_preds_batch = test_preds_batch / counter   
        test_preds.append(test_preds_batch)

    test_preds = np.concatenate(test_preds)

FileNotFoundError: [Errno 2] No such file or directory: '../input/isic-scd-resnet18-train/models/fold_1/model_resnet18_v1_epoch_0/pytorch_model_1.bin'

In [ ]:
test_predictions_df.head()

In [ ]:
test_predictions_df[target_column] = test_predictions_df[[f"fold_{fold}" for fold in all_folds]].mean(axis=1)

In [ ]:
test_predictions_df.head()

In [ ]:
test_predictions_df[target_column].describe()

In [ ]:
test_predictions_df[[id_column, target_column]].head(10)

In [ ]:
test_predictions_df[[id_column, target_column]].to_csv("submission.csv", index=False)